<a href="https://colab.research.google.com/github/muhammadmada/diseasespreadsim/blob/master/disease_spread_simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Disease Spread Simulation Project
**Pre-Alpha Proof of Concept Demo for VEKTORPROJEKT**

This notebook is a pre-alpha demonstration for an interactive disease spread simulation with codename VEKTORPROJEKT. 

Updates will follow. 

###Initialization

**Load modules and libraries**




In [ ]:
from scipy.integrate import odeint
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
%matplotlib inline
!pip install mpld3
import mpld3
mpld3.enable_notebook()

     |████████████████████████████████| 1.0MB 2.8MB/s 
  Created wheel for mpld3: filename=mpld3-0.5.1-cp36-none-any.whl size=364064 sha256=5cf502129c756ea54a7220ea59a514075b48a2543546fbb3a3de03d7d11c4334
  Stored in directory: /root/.cache/pip/wheels/38/68/06/d119af6c3f9a2d1e123c1f72d276576b457131b3a7bf94e402
Successfully built mpld3


**Define matplotlib plotting**

In [ ]:
def modseirplot(t, S, Sq, E, Eq, I, H, R, L=None, D=None, R0=None, Alpha=None, CFR=None):
  f, ax = plt.subplots(1,1, figsize=(10,4))
  ax.plot(t, S, 'b', alpha=0.7, linewidth=2, label= 'Susceptible') #t=time; label/var;color/styling;transp#
  ax.plot(t, Sq, 'b--', alpha=0.7, linewidth=2, label= 'Quarantined Susceptible')
  ax.plot(t, E, 'y', alpha=0.7, linewidth=2, label= 'Exposed')
  ax.plot(t, Eq, 'y--', alpha=0.7, linewidth=2, label= 'Quarantined Exposed')
  ax.plot(t, I, 'r', alpha=0.7, linewidth=2, label= 'Infected')
  ax.plot(t, H, 'g--', alpha=0.7, linewidth=2, label= 'Hospitalized')
  ax.plot(t, R, 'g', alpha=0.7, linewidth=2, label= 'Recovered')
  if D is not None:
      ax.plot(t, D, 'k', alpha=0.7, linewidth=2, label='Dead')
      ax.plot(t, S+Sq+E+Eq+I+H+R+D, 'c--', alpha=0.7, linewidth=2, label='Total')
  else:
       ax.plot(t, S+Sq+E+Eq+I+H+R+D, 'c--', alpha=0.7, linewidth=2, label='Total')
    
  ax.set_xlabel('Time (days)')
  ax.yaxis.set_tick_params(length=0)
  ax.xaxis.set_tick_params(length=0)
  ax.grid(b=True, which='major', c='w', lw=2, ls='-')
  legend = ax.legend(borderpad=2.0)
  legend.get_frame().set_alpha(0.5)
  for spine in ('top','right', 'bottom', 'left'):
      ax.spines[spine].set_visible(False)
  if L is not None:
     plt.title("Lockdown after {} days".format(L))
  plt.show()
  if R0 is not None or CFR is not None:
   f = plt.figure(figsize=(12,4))
  
  if R0 is not None:
    # sp1
    ax1 = f.add_subplot(121)
    ax1.plot(t, R0, 'b--', alpha=0.7, linewidth=2, label='R_0')

    ax1.set_xlabel('Time (days)')
    ax1.title.set_text('R_0 over time')
    # ax.set_ylabel('Number (1000s)')
    # ax.set_ylim(0,1.2)
    ax1.yaxis.set_tick_params(length=0)
    ax1.xaxis.set_tick_params(length=0)
    ax1.grid(b=True, which='major', c='w', lw=2, ls='-')
    legend = ax1.legend()
    legend.get_frame().set_alpha(0.5)
    for spine in ('top', 'right', 'bottom', 'left'):
      ax.spines[spine].set_visible(False)

  if Alpha is not None:
    # sp2
    ax2 = f.add_subplot(122)
    ax2.plot(t, Alpha, 'r--', alpha=0.7, linewidth=2, label='alpha')

    ax2.set_xlabel('Time (days)')
    ax2.title.set_text('fatality rate over time')
    # ax.set_ylabel('Number (1000s)')
    # ax.set_ylim(0,1.2)
    ax2.yaxis.set_tick_params(length=0)
    ax2.xaxis.set_tick_params(length=0)
    ax2.grid(b=True, which='major', c='w', lw=2, ls='-')
    legend = ax2.legend()
    legend.get_frame().set_alpha(0.5)
    for spine in ('top', 'right', 'bottom', 'left'):
      ax.spines[spine].set_visible(False)

    plt.show();

##Simulation Initialization

**Opening required files**

In [ ]:
#Opening necessary csv/json document
cov_url= ' ' #githubsuercontent csv file
covid_df = pd.read_csv(cov_url, index_col=0)
pop_url = ' '#githubusercontent csv file
pop_df = pd.read_csv(pop_url, index_col=0)

FileNotFoundError: ignored

**Loading general simulation equation**

In [ ]:
#Defining derivative equation part
def deriv(y, t, N, Beta, DeltaI, DeltaQ, GammaI, GammaH):
    S, Sq, E, Eq, I, H, R, D = y
    dSdt = int(-(Beta*c + c*q*(1-Beta)*S*(I+Theta+E)+ Lambda*Sq))
    dSqdt = int((1-Beta)*c*q*S*(I+Theta*E)-Lambda*Sq)
    dEdt = int(Beta*c*(1-q)*S*(I+Theta*E)-Sigma*E)
    dEqdt = int(Beta*c*q*S*(I+Theta*E)-DeltaQ*Eq)
    dIdt = int(Sigma*E-(deltaI+Alpha+GammaI)*I)
    dHdt = int(deltaI*I+DeltaQ*Eq-(Alpha+GammaH)*H)
    dRdt = int(GammaI*I+GammaH*H)
    dDdt = Alpha*(I+H)
    return dSdt, dSqdt, dEdt, dEqdt, dIdt, dHdt, dRdt, dDdt
#Model params and data(Some datas need to be connected with mongodb yes)
c = float(3.6)
Beta = (6.93*10**-11) #Equals to R_0*GammaI
Gamma = int(1) #Refers to plain Gamma from disease, distributed to derivative Gammas
Inf =  123#Infection period(Days)
Inc = 2141#Incubation period(Days)
DeltaI = float(0.13) #1/days of infection, infected
DeltaQ = float(0.13) #1/days of infection, quarantined
GammaI = float(0.003) #1/days of incubation, infected
GammaH = float(0.009) #1/days of incubation, hospitalized
Q = float(9*10**-7) #Quarantine ratio
Alpha = float(0.0001) #Death rate
Theta = float(0.6) #Ratio of transmission
Lambda = float(1/14) #1/quarantine days
T = float(40) #Simulation timespan, maybe add progressing time function with starting date
t = float(0.1) #Forgot what this means
NN = float(T/t) #Same with this
N = 2 #Population in a region
L = 133 # Lockdown period (Days), may add date implemented + date lifted function


**Regional spreading simulation**

**Combine both simulation**

###Mapping Initialization

**Load mapping libraries and modules**

In [ ]:
import geopandas as gpd


**Define mapping and map overlay function**

###Data Plotting (Graph)

**Load additional plotting libraries** 

In [ ]:
import plotly.plotly as py
import numpy as np
from plotly.offline import init_notebook_mode, iplot

##Simulation Generation

**Generating simulation CSV files**

**Generating animated and mapping simulation**

###Exporting Results

**Exporting results into downloadable data (video, PDF, CSVs)**

In [ ]:
#Exporting into video
HTML(display().to_html5_video())
#Exporting into CSVs
#Exporting into PDF

**Exporting results into HTML**

###Export/Import from Database

**Export result into DB**

**Import result from DB**

**Show result from import**

###Notes

#####References